In [ ]:
import pandas as pd


SUBIR A BASE DE DADOS E VERIFICAR SE EXISTEM DADOS NULOS


In [ ]:
df = pd.read_csv('fraud_dataset.csv')
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [ ]:
df.isnull().sum()


distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [ ]:
df.shape

(1000000, 8)

In [ ]:
print(df.columns)

Index(['distance_from_home', 'distance_from_last_transaction',
       'ratio_to_median_purchase_price', 'repeat_retailer', 'used_chip',
       'used_pin_number', 'online_order', 'fraud'],
      dtype='object')


VERIFICAÇÃO DE DESEQUILIBRIO DE FRAUDES

In [ ]:
round(df['fraud'].value_counts(normalize=True)*100,2)

fraud
0.0    91.26
1.0     8.74
Name: proportion, dtype: float64

SEPARAÇÃO DA BASE EM TREINO E TESTE - APLICAÇÃO DO METODO HOLD OUT (80% TREINO 20% TESTE)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('fraud', axis=1)
Y = df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

(800000, 7) (200000, 7)


PADRONIZAÇÃO DAS VARIAÁVEIS NÚMERICAS

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_escalonado = scaler.transform(X_train)
X_test_escalonado = scaler.transform(X_test)

APLICAÇÃO DO OVERSAMPLING

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_escalonado, y_train)
#

In [ ]:
print(pd.Series(y_train_resampled.value_counts(normalize=True))*100)

fraud
0.0    50.0
1.0    50.0
Name: proportion, dtype: float64


REGRESSÃO LOGISTICA E VALIDAÇÃO CRUZADA

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(penalty='l2')


#validação cruzada dos dados
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(modelo, X_train_resampled, y_train_resampled, cv=cv, scoring='roc_auc')


In [ ]:
print(scores.mean())

0.9793964761399654


In [ ]:
modelo.fit(X_train_resampled, y_train_resampled)
y_pred = modelo.predict(X_test_escalonado)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(auc_score.mean())

              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

0.9408226521021739
